#### Import all the dependencies

In [28]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pymongo
import pandas as pd
from pprint import pprint
import json

#### Initialize PyMongo to work with MongoDBs

In [10]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#### Initialize the variables to be used by splinter.

In [11]:
# #Initialize the variables to be used by splinter.
# executable_path = {'executable_path': 'C:/UCB/chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=True)

#### Create MongoDB Database and collection

In [12]:
# Define database and collection
db = client.Mars_db
db.MarsDataCollection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Mars_db'), 'MarsDataCollection')

#### Browse the URL to be scraped and create BeautifulSoup object

In [29]:
# URL of page to be scraped
url = 'https://mars.nasa.gov'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')


### NASA Mars News

Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [27]:
# Finding NASA Mars News - scraping and collecting news_title and news_p

results = soup.find_all('ul', class_='item_list list_view')

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of Latest news
        anchortag = result.find('div', class_='list_text').find('a')
        news_title = anchortag.text
        print(news_title)
               
        # Identify and return link to Latest news item
        partiallink = anchortag['href']
        
        #Form the URL to the latest news 
        newsurl = url+partiallink
        
        # use Splinter to browse to the news page
        #Initialize the variables to be used by splinter.
        executable_path = {'executable_path': 'C:/UCB/chromedriver.exe'}
        browser = Browser('chrome', **executable_path, headless=True)
        browser.visit(newsurl)
        
        #obtain the html of the page
        newsresp = requests.get(newsurl)
                  
        # Create BeautifulSoup object; parse with 'lxml'
        news_soup = BeautifulSoup(newsresp.text, 'lxml')
        news_results = news_soup.find_all('div', class_="wysiwyg_content")
        for news in news_results:
            
            ptags = news.find_all('p')
          
            for ptag in ptags:
                
                if ptag.text:
                    news_p = ptag.text
                    print(news_p)
                    break
                else:
                    continue
            break
 
    except Exception as e:
        print(e)


InSight Captures Sunrise and Sunset on Mars
A camera on the spacecraft's robotic arm snapped the photos on April 24 and 25, the 145th Martian day, or sol, of the mission. In local Mars time, the shots were taken starting around 5:30 a.m. and then again starting around 6:30 p.m. As a bonus, a camera under the lander's deck also caught clouds drifting across the Martian sky at sunset.


### JPL Mars Space Images - Featured Image




In [92]:
# Visit the url for JPL Featured Space Image here.

JPLurl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

#Use splinter to navigate the site and find the image url for the current Featured Mars Image and 
#assign the url string to a variable

browser.visit(JPLurl)
JPLresp = requests.get(JPLurl)  

# Create BeautifulSoup object; parse with 'lxml'
JPL_soup = BeautifulSoup(JPLresp.text, 'lxml')

FullImgbuttontag= JPL_soup.find('a', class_="button fancybox")

In [93]:
# Constructing the URL to the Image article
spaceurl = (browser.url).split("?")[0]
partialURL = FullImgbuttontag['data-link'].split("spaceimages/")[1]
imgartURL = spaceurl+partialURL

print(imgartURL)

https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA19382


In [94]:
browser.visit(imgartURL)
imgresp = requests.get(imgartURL)  

# Create BeautifulSoup object; parse with 'lxml'
img_soup = BeautifulSoup(imgresp.text, 'lxml')

#### JPL Mars Space Images - Featured Image

In [95]:
figtag = img_soup.find('figure', class_="lede")
imgtaghref = figtag.find('a')['href'].split("/spaceimages/")[1]

featured_image_url = spaceurl+imgtaghref

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19382_hires.jpg


#### Mars Weather

In [119]:
marstwitterurl = "https://twitter.com/marswxreport?lang=en"
tweetresp = requests.get(marstwitterurl)  

tweet_soup = BeautifulSoup(tweetresp.text, 'lxml')


for eachlitag in tweet_soup.find_all('li', class_="stream-item"):      # Loop through all tweets
    
    contentdiv = eachlitag.find('div', class_="content")                    # content div
    divstreamheader = contentdiv.find('div', class_="stream-item-header")   #finding stream-item-header to identify the user
    atagdivstream = divstreamheader.find('a')
 
    if atagdivstream['href'] =='/MarsWxReport':                                   # Filtering tweet from MarsWxReport
        ptagtweet = contentdiv.find('div', class_="js-tweet-text-container").find('p')  # find the ptag with the tweetcontent
        ptagtweet.a.decompose()                                     # remove the anchor tag with the <p> tag
        mars_weather = ptagtweet.text
        break
    else:
        continue
    
 

InSight sol 155 (2019-05-04) low -99.3ºC (-146.8ºF) high -18.8ºC (-1.8ºF)
winds from the SW at 4.5 m/s (10.1 mph) gusting to 14.3 m/s (31.9 mph)
pressure at 7.40 hPa


### Mars Facts

In [46]:
factsurl = "https://space-facts.com/mars/"

#use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
tables = pd.read_html(factsurl)

factsdf = tables[0]
factsdf.columns = ["Description" , "Values"]

In [47]:
# display the content of the DataFrame
factsdf.head()

,Description,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [6]:
#Use Pandas to convert the data to a HTML table string.
MarsFacts_table=factsdf.to_html()

In [48]:
factsdf.reset_index(drop=True, inplace=True)

In [52]:
marsfactsdesc = factsdf["Description"].tolist()
marsfactsvalues= factsdf["Values"].tolist()

In [53]:
print(marsfacts)

In [43]:
print(marsfactsvalues)

['6,792 km', '6,752 km', '6.42 x 10^23 kg (10.7% Earth)', '2 (Phobos & Deimos)', '227,943,824 km (1.52 AU)', '687 days (1.9 years)', '-153 to 20 °C', '2nd millennium BC', 'Egyptian astronomers']


### Mars Hemispheres

In [129]:
#Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [130]:
hemi_result = requests.get(hemi_url)
hemi_soup = BeautifulSoup(hemi_result.text, "lxml")


In [149]:
# scraping the data
resultitems = hemi_soup.find('div' , class_="collapsible results").find_all('div', class_="item")

hemisphere_image_urls =[]
eachhemisphere = {}

Astrourldomain = hemi_url.split("/search")[0]

for item in resultitems:
    title = item.find('h3').text
    
    itempartialurl = item.find('a')['href']
    #url to browse
    itemurl = Astrourldomain+itempartialurl
    
    # browse to the url
    browser.visit(itemurl)
    hemi_item_result = requests.get(itemurl)
    hemi_item_soup = BeautifulSoup(hemi_item_result.text, "lxml")
    
    aimgtag = hemi_item_soup.find('div', class_="downloads").find('a')
    img_url = aimgtag['href']
   
    eachhemisphere = {"title":title, "img_url":img_url}
    hemisphere_image_urls.append(eachhemisphere)

pprint(hemisphere_image_urls)

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]
